In [5]:
%cd C:\mimic-iv-2.2\mimic-iv-2.2\hosp

C:\mimic-iv-2.2\mimic-iv-2.2\hosp


In [6]:
import pandas as pd

In [7]:
df_d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv')
df_diagnoses_icd = pd.read_csv("diagnoses_icd.csv")


In [9]:

# 1.전체에서 암icd코드 있는 행들의 subject_id를 중복제거하고 추출 ==>암있는 환자들만 나옴 
# 2. 암있는 환자들중 icd_code가 고혈압인 행들만 골라서 그 subject_id추출한다.==> 암있고 고혈압인 subject_id만 추출가능.
# 3. .seq_num==1 인 애들중  코드가 암인 애들만 추출
# 4. 다른 테이블들이랑 병합.



# df_cancer = df_diagnoses_icd[df_diagnoses_icd['icd_code'].str.match(r'^C[0-9]{2}|^D0[0-9]')]
# cancer_id = df_cancer['subject_id'].value_counts().index

# df_merge1 = pd.merge(df_diagnoses_icd, df_d_icd_diagnoses, on=['icd_code'])
# df_merge1
# df_a = df_merge1[df_merge1['subject_id'].isin(cancer_id)]
# mask_bool = df_a['icd_code'].str.match(r'^I1[0-5]$|^401')
# cancer_highbp = df_a.loc[mask_bool]['subject_id'].value_counts().index
# df_cancer_highbp= df_merge1[df_merge1['subject_id'].isin(cancer_highbp)]
# df_cancer_highbp = df_cancer_highbp[df_cancer_highbp['seq_num'] == 1]
# df_cancer_highbp[df_cancer_highbp['icd_code'].str.match(r'^C[0-9]{2}|^D0[0-9]')]

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
1601378,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck
1601381,11282936,22922093,1,C675,10,10,Malignant neoplasm of bladder neck
1601385,13668295,25846146,1,C675,10,10,Malignant neoplasm of bladder neck
1601387,14449673,27244589,1,C675,10,10,Malignant neoplasm of bladder neck
1601391,16455035,29863248,1,C675,10,10,Malignant neoplasm of bladder neck
...,...,...,...,...,...,...,...
4859583,18904293,25785479,1,C8174,10,10,"Other Hodgkin lymphoma, lymph nodes of axilla ..."
4859732,19128927,28814179,1,C44692,10,10,Other specified malignant neoplasm of skin of ...
4859827,19282415,28695316,1,C004,10,10,"Malignant neoplasm of lower lip, inner aspect"
4859828,19290501,28187167,1,C8468,10,10,"Anaplastic large cell lymphoma, ALK-positive, ..."


In [8]:
import pandas as pd

# 고혈압 진단 코드 정규식
high_bp_regex = r'^I1[0-5]$|^401'

#암 진단 코드 정규식
cancer_regex = r'^C[0-9]{2}|^D0[0-9]'


# 암 진단 코드 추출
cancer_df = df_diagnoses_icd[df_diagnoses_icd['icd_code'].str.match(cancer_regex)]

# 암 진단을 받은 환자 ID 추출
cancer_patient_ids = cancer_df['subject_id'].unique()

# 암 진단을 받은 환자들의 데이터 추출
cancer_diagnoses_merged = pd.merge(df_diagnoses_icd, df_d_icd_diagnoses, on=['icd_code'])
cancer_patients_data = cancer_diagnoses_merged[cancer_diagnoses_merged['subject_id'].isin(cancer_patient_ids)]



# 암 및 고혈압을 동시에 가진 환자들의 데이터 추출 (첫 번째 진단만 포함)
cancer_high_bp_ids = cancer_patients_data[cancer_patients_data['icd_code'].str.match(high_bp_regex)]['subject_id'].unique()
cancer_high_bp_df = cancer_diagnoses_merged[(cancer_diagnoses_merged['subject_id'].isin(cancer_high_bp_ids)) & 
                                            (cancer_diagnoses_merged['seq_num'] == 1) & 
                                            (cancer_diagnoses_merged['icd_code'].str.match(cancer_regex))]

cancer_high_bp_df

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
1601378,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck
1601381,11282936,22922093,1,C675,10,10,Malignant neoplasm of bladder neck
1601385,13668295,25846146,1,C675,10,10,Malignant neoplasm of bladder neck
1601387,14449673,27244589,1,C675,10,10,Malignant neoplasm of bladder neck
1601391,16455035,29863248,1,C675,10,10,Malignant neoplasm of bladder neck
...,...,...,...,...,...,...,...
4859583,18904293,25785479,1,C8174,10,10,"Other Hodgkin lymphoma, lymph nodes of axilla ..."
4859732,19128927,28814179,1,C44692,10,10,Other specified malignant neoplasm of skin of ...
4859827,19282415,28695316,1,C004,10,10,"Malignant neoplasm of lower lip, inner aspect"
4859828,19290501,28187167,1,C8468,10,10,"Anaplastic large cell lymphoma, ALK-positive, ..."
